# **Train**

In [1]:
!pip install lmdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 5.9 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/wilwil2468/IRSRCNN.git

Cloning into 'IRSRCNN'...
remote: Enumerating objects: 12542, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 12542 (delta 99), reused 165 (delta 51), pack-reused 12326 (from 4)
Receiving objects: 100% (12542/12542), 1.12 GiB | 50.66 MiB/s, done.
Resolving deltas: 100% (209/209), done.
Updating files: 100% (12012/12012), done.


In [3]:
%cd /kaggle/working/IRSRCNN

/kaggle/working/IRSRCNN


In [4]:
!git pull

Already up to date.


For “915” (k9–k1–k5) you used hr_crop_size = 21

For “935” (k9–k3–k5) you used hr_crop_size = 19

For “955” (k9–k5–k5) you used hr_crop_size = 17

In [5]:
!python precompute_patches.py dataset/train train_patches_955.lmdb \
    --hr 17 --lr 33 --per-image 100

Dumped 1074200 patches to train_patches_955.lmdb


In [6]:
!python precompute_patches.py dataset/validation valid_patches_955.lmdb \
    --hr 17 --lr 33 --per-image 100

Dumped 114400 patches to valid_patches_955.lmdb


In [7]:
# train SRCNN-955 model
!rm -rf dataset/*.npy
!python train.py  --steps=300000                    \
                  --architecture="955"              \
                  --batch_size=128                \
                  --save-best-only=1                \
                  --save-every=1000                 \
                  --save-log=0                      \
                  --ckpt-dir="checkpoint/SRCNN955"  \
                  --num-worker=4

/kaggle/working/IRSRCNN/model.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.ckpt_man = torch.load(ckpt_path)
/kaggle/working/IRSRCNN/model.py:53: FutureWarning: 

# **Test**

In [8]:
# Test on Set5
!python test.py --scale=2 --architecture=955 --ckpt-path="default"
!python test.py --scale=3 --architecture=955 --ckpt-path="default"
!python test.py --scale=4 --architecture=955 --ckpt-path="default"

/kaggle/working/IRSRCNN/model.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(filepath, map_location=torch.device(self.device))
11.032090759277343
/kaggle/wo

In [9]:
# Test on FLIR data
!python test-ir.py --scale=2 --architecture=955 --ckpt-path="default"
!python test-ir.py --scale=3 --architecture=955 --ckpt-path="default"
!python test-ir.py --scale=4 --architecture=955 --ckpt-path="default"

/kaggle/working/IRSRCNN/model.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(filepath, map_location=torch.device(self.device))
12.041500091552734
/kaggle/wo

# **Demo**

In [10]:
!python demo.py --image-path="dataset/test1.png" \
                --architecture="955"             \
                --ckpt-path="default"            \
                --scale=4

/kaggle/working/IRSRCNN/model.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(filepath, map_location=torch.device(self.device))
